In [2]:
%cd /workspace
import pytorchvideo
import pytorchvideo.layers
from pytorchvideo.layers.distributed import cat_all_gather
%cd slowfast
import os
import cv2
import torch
import slowfast
import imutils
from imutils.video import FPS
import numpy as np
from tqdm import tqdm
import logging
logging.basicConfig(format='%(asctime)s - p%(process)s {%(filename)s:%(lineno)d} - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from ultralytics import YOLO
from slowfast.config.defaults import get_cfg, assert_and_infer_cfg
from slowfast.models import build_model
from slowfast.utils.checkpoint import load_checkpoint
from slowfast.utils.parser import load_config, parse_args
from slowfast.utils.misc import launch_job
from slowfast.visualization.utils import process_cv2_inputs
from slowfast.visualization.demo_loader import ThreadVideoManager, VideoManager
from slowfast.visualization.predictor import ActionPredictor
from slowfast.visualization.video_visualizer import VideoVisualizer
import slowfast.utils.misc as misc
from slowfast.datasets import loader
from slowfast.datasets.utils import get_sequence
from slowfast.datasets.cv2_transform import scale, scale_boxes

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)

/workspace
/workspace/slowfast


/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
ava_classes = ['bend/bow (at the waist)','crawl','crouch/kneel','dance','fall down',
 'get up','jump/leap','lie/sleep','martial art','run/jog','sit','stand','swim','walk',
 'answer phone','brush teeth','carry/hold (an object)','catch (an object)','chop','climb (e.g., a mountain)',
 'clink glass','close (e.g., a door, a box)','cook','cut','dig','dress/put on clothing','drink',
 'drive (e.g., a car, a truck)','eat','enter','exit','extract','fishing','hit (an object)','kick (an object)',
 'lift/pick up','listen (e.g., to music)','open (e.g., a window, a car door)','paint','play board game',
 'play musical instrument','play with pets','point to (an object)','press','pull (an object)',
 'push (an object)','put down','read','ride (e.g., a bike, a car, a horse)','row boat','sail boat','shoot',
 'shovel','smoke','stir','take a photo','text on/look at a cellphone','throw','touch (an object)',
 'turn (e.g., a screwdriver)','watch (e.g., TV)','work on a computer','write','fight/hit (a person)',
 'give/serve (an object) to (a person)','grab (a person)','hand clap','hand shake', 'hand wave',
 'hug (a person)','kick (a person)','kiss (a person)','lift (a person)','listen to (a person)',
 'play with kids','push (another person)','sing to (e.g., self, a person, a group)',
 'take (an object) from (a person)','talk to (e.g., self, a person, a group)','watch (a person)']

action_class_list = ['fall',
 'standing',
 'sitting',
 'walking',
 'crawling',
 'sleeping',
 'eating',
 'jumping',
 'others']

In [4]:
yolo_weights = "../yolov8m_no_augmentation.pt"
yolov8_model = YOLO(yolo_weights)
yolov8_model = yolov8_model.to('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Config:
    def __init__(self, shard_id=0, num_shards=1, init_method="tcp://localhost:9999", cfg_files=None, opts=None):
        self.shard_id = shard_id
        self.num_shards = num_shards
        self.init_method = init_method
        self.cfg_files = cfg_files if cfg_files is not None else []
        self.opts = opts #if opts is not None else []

    def update(self, shard_id=None, num_shards=None, init_method=None, cfg_files=None, opts=None):
        if shard_id is not None:
            self.shard_id = shard_id
        if num_shards is not None:
            self.num_shards = num_shards
        if init_method is not None:
            self.init_method = init_method
        if cfg_files is not None:
            self.cfg_files = cfg_files
        if opts is not None:
            self.opts = opts

    def display(self):
        print("Configuration:")
        print(f"  Shard ID: {self.shard_id}")
        print(f"  Number of Shards: {self.num_shards}")
        print(f"  Initialization Method: {self.init_method}")
        print(f"  Configuration Files: {self.cfg_files}")
        print(f"  Additional Options: {self.opts}")
 
config_file = "configs/AVA/c2/SLOWFAST_32x2_R101_50_50_v2.1.yaml"
parse_config = Config(cfg_files=[config_file])
parse_config.display()
args = parse_config

# cfg = load_config(args, "../slowfast_config_daycare_2.yaml")
# cfg.TEST.CHECKPOINT_FILE_PATH = "checkpoints/checkpoint_epoch_00050.pyth"
cfg = load_config(args, config_file)
cfg.TEST.CHECKPOINT_FILE_PATH = "SLOWFAST_32x2_R101_50_50.pkl"
cfg.NUM_GPUS = 1
cfg.DETECTION.ENABLE = True
cfg.TEST.NUM_TEMPORAL_CLIPS = [cfg.TEST.NUM_ENSEMBLE_VIEWS]
cfg = assert_and_infer_cfg(cfg)
action_model = build_model(cfg)
action_model.eval()

import slowfast.utils.checkpoint as cu
import slowfast.utils.distributed as du
# Set up environment.
du.init_distributed_training(cfg)
# Set random seed from configs.
np.random.seed(cfg.RNG_SEED)
torch.manual_seed(cfg.RNG_SEED)

flops, params = misc.log_model_info(action_model, cfg, use_train_input=False)
cu.load_test_checkpoint(cfg, action_model)



Configuration:
  Shard ID: 0
  Number of Shards: 1
  Initialization Method: tcp://localhost:9999
  Configuration Files: ['configs/AVA/c2/SLOWFAST_32x2_R101_50_50_v2.1.yaml']
  Additional Options: None
path_to_config : configs/AVA/c2/SLOWFAST_32x2_R101_50_50_v2.1.yaml


2025-06-19 14:01:49,103 - p153239 {misc.py:186} - INFO - Model:
SlowFast(
  (s1): VideoModelStem(
    (pathway0_stem): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (pathway1_stem): ResNetBasicStem(
      (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
      (bn): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (pool_layer): MaxPool3d(kernel_size=[1, 3, 3], stride=[1, 2, 2], padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
  )
  (s1_fuse): FuseFastToSlow(
    (conv_f2s): Conv3d(8, 16, kernel_size=(5, 1, 1), stride=(4, 1, 1), padding=(2, 0, 0), bias=

Thu Jun 19 14:01:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:01:01.0 Off |                    0 |
| N/A   58C    P0             98W /  250W |   25111MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

2025-06-19 14:01:53,765 - p153239 {checkpoint.py:213} - INFO - Loading network weights from SLOWFAST_32x2_R101_50_50.pkl.


missing keys: []
unexpected keys: []


In [6]:
num_frames = 32
sampling_rate = 2


def draw_text(img, text,
          pos=(0, 0),
          font=cv2.FONT_HERSHEY_PLAIN,
          font_scale=3,
          text_color=(0, 255, 0),
          font_thickness=2,
          text_color_bg=(0, 0, 0)
          ):

    x, y = pos
    # font_scale = 1
    # font = cv2.FONT_HERSHEY_PLAIN
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, (x, y - text_h - 10), (x + text_w + 10, y), text_color_bg, -1)
    cv2.putText(img, text, (x+5, y-5), font, font_scale, text_color, font_thickness)




def draw_bb_text(frame, text, bbox,
                 font=cv2.FONT_HERSHEY_PLAIN,
                 base_font_scale=3,
                 text_color=(0, 255, 0),
                 base_font_thickness=2,
                 text_color_bg=(255, 255, 255)):
    # Get frame dimensions
    frame_h, frame_w = frame.shape[:2]

    # Calculate scaling factors
    scale_factor = max(frame_w, frame_h) / 500  # Normalize scaling to a base size (500px)
    font_scale = base_font_scale * scale_factor
    font_thickness = max(1, int(base_font_thickness * scale_factor))
    padding = max(2, int(4 * scale_factor))  # Additional padding around text

    # Extract bounding box coordinates
    startX, startY, endX, endY = bbox

    # Get text size
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size

    # Adjust text box height and padding
    tboxh = text_h + padding * 2

    # Ensure coordinates stay within bounds
    startY = max(tboxh, startY)
    startX = max(1, startX)

    # Create background for text
    bg = np.ones_like(frame[startY-tboxh:startY, startX-1:startX+text_w+3]).astype('uint8') * 255
    bg[:, :] = text_color_bg
    frame[startY-tboxh:startY, startX-1:startX+text_w+3] = cv2.addWeighted(
        frame[startY-tboxh:startY, startX-1:startX+text_w+3],
        0.0, bg, 1.0, 1
    )

    # Draw the text
    # print(f"font ; {font}, font_scale : {font_scale}, font_thickness : {font_thickness}")
    cv2.putText(frame, text, (startX, startY-padding),
                font, font_scale, text_color, font_thickness)


def get_color_from_id(idx):
    idx = idx * 3
    color = (int((37 * idx) % 255), int((17 * idx) % 255), int((29 * idx) % 255))

    return color

def isLightOrDark(rgbColor=[0,128,255]):
    [r,g,b]=rgbColor
    hsp = np.sqrt(0.299 * (r * r) + 0.587 * (g * g) + 0.114 * (b * b))
    if (hsp>127.5):
        return [0, 0, 0]
    else:
        return [255, 255, 255]


def format_tracking_results(tracking_results, target_classes=None, classlist = ["adult","child","mobile","table","bookstand","chair","chairs", "hurdle", "crib", "book", "food"]):
    for tracking_result in tracking_results:
        bboxes = tracking_result.boxes.cpu().numpy()
        tracking_results = []
        for xyxy, tid, class_id in zip(bboxes.xyxy, bboxes.id, bboxes.cls):
            center_x = int((xyxy[0] + xyxy[2]) / 2)
            center_y = int((xyxy[1] + xyxy[3]) / 2)
            if target_classes is None:
                tracking_results.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), int(tid), classlist[int(class_id)], center_x, center_y])
            elif class_id in target_classes:
                tracking_results.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), int(tid), classlist[int(class_id)], center_x, center_y])
        return tracking_results

def format_detection_results(detection_results, target_classes=None, classlist = ["adult","child","mobile","table","bookstand","chair","chairs", "hurdle", "crib", "book", "food"]):
    for detection_result in detection_results:
        bboxes = detection_result.boxes.cpu().numpy()
        detection_results = []
        for xyxy, class_id in zip(bboxes.xyxy, bboxes.cls):
            center_x = int((xyxy[0] + xyxy[2]) / 2)
            center_y = int((xyxy[1] + xyxy[3]) / 2)
            if target_classes is None:
                detection_results.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), classlist[int(class_id)], center_x, center_y])
            elif class_id in target_classes:
                detection_results.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3]), classlist[int(class_id)], center_x, center_y])
        return detection_results
